# BatchBacktesting

In [1]:
# import library
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
import sys
import os
import httpx

In [3]:
from src.strategies import *
import concurrent.futures
from datetime import datetime
import glob
import warnings
from rich.progress import track
warnings.filterwarnings("ignore")

In [4]:
def make_api_request(api_endpoint, params):
    with httpx.Client() as client:
        # Make the GET request to the API
        response = client.get(api_endpoint, params=params)
        if response.status_code == 200:
            return response.json()
        print("Error: Failed to retrieve data from API")
        return None


In [5]:
BASE_URL_FMP = "https://financialmodelingprep.com/api/v3"
BASE_URL_BINANCE = "https://fapi.binance.com/fapi/v1/"

FMP_API_KEY = "17c09553207c6d6e7bab10003e604aa8"
BINANCE_API_KEY = "17754325821w1b1e2khd34561hc54hh1"


def get_historical_price_full_crypto(symbol):
    api_endpoint = f"{BASE_URL_FMP}/historical-price-full/crypto/{symbol}"
    params = {"apikey": FMP_API_KEY}
    return make_api_request(api_endpoint, params)


def get_historical_price_full_stock(symbol):
    api_endpoint = f"{BASE_URL_FMP}/historical-price-full/{symbol}"
    params = {"apikey": FMP_API_KEY}

    return make_api_request(api_endpoint, params)

def get_SP500():
    api_endpoint = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    data = pd.read_html(api_endpoint)
    return list(data[0]['Symbol'])

def get_all_crypto():
    """
    All possible crypto on binance
    """

    api_endpoint = "https://api.binance.com/api/v3/ticker/price"
    data = make_api_request(api_endpoint, {})
    return [i['symbol'] for i in data]


def get_financial_statements_lists():
    api_endpoint = f"{BASE_URL_FMP}/financial-statement-symbol-lists"
    params = {"apikey": FMP_API_KEY}
    return make_api_request(api_endpoint, params)

In [6]:
def run_backtests_strategies(instruments, strategies):
    """
    Run backtests for a list of instruments using a specified strategy.

    Args:
        instruments (list): List of instruments to run backtests for
        strategies (list): List of strategies to run backtests for

    Returns:
        List of outputs from run_backtests()

    """

    # find strategies in the STRATEGIES
    strategies = [x for x in STRATEGIES if x.__name__ in strategies]
    outputs = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for strategy in strategies:
            future = executor.submit(run_backtests, instruments, strategy, 4)
            futures.append(future)

        for future in concurrent.futures.as_completed(futures):
            outputs.extend(future.result())

    return outputs

def check_crypto(instrument):
    """
    Check if the instrument is crypto or not
    """
    return instrument in get_all_crypto()

def check_stock(instrument):
    """
    Check if the instrument is crypto or not
    """
    return instrument not in get_financial_statements_lists()


def process_instrument(instrument, strategy):
    """
    Process a single instrument for a backtest using a specified strategy.
    Returns a Pandas dataframe of the backtest results.
    """
    try:

        if check_crypto(instrument):
            data = get_historical_price_full_crypto(instrument)
        else:
            data = get_historical_price_full_stock(instrument)

        if data is None or "historical" not in data:
            print(f"Error processing {instrument}: No data")
            return None

        data = clean_data(data)

        bt = Backtest(
            data, strategy=strategy, cash=100000, commission=0.002, exclusive_orders=True
        )
        output = bt.run()
        output = process_output(output, instrument, strategy)
        return output, bt
    except Exception as e:
        print(f"Error processing {instrument}: {str(e)}")
        return None


def process_instrument_optimise(instrument, strategy):
    """
    Process a single instrument for a backtest using a specified strategy.
    Returns a Pandas dataframe of the backtest results.
    """
    try:
        data = get_historical_price_full_stock(instrument)
        data = clean_data(data)
        bt = Backtest(
            data, strategy=strategy, cash=100000, commission=0.002, exclusive_orders=True
        )
        output = bt.optimize_func()
        output = process_output(output, instrument, strategy)
        return output, bt
    except Exception as e:
        print(f"Error processing {instrument}: {str(e)}")
        return None


def clean_data(data):
    """
    Clean historical price data for use in a backtest.
    Returns a Pandas dataframe of the cleaned data.
    """
    data = data["historical"]
    data = pd.DataFrame(data)
    data.columns = [x.title() for x in data.columns]
    data = data.drop(
        [
            "Adjclose",
            "Unadjustedvolume",
            "Change",
            "Changepercent",
            "Vwap",
            "Label",
            "Changeovertime",
        ],
        axis=1,
    )
    data["Date"] = pd.to_datetime(data["Date"])
    data.set_index("Date", inplace=True)
    data = data.iloc[::-1]
    return data


def process_output(output, instrument, strategy, in_row=True):
    """
    Process backtest output data to include instrument name, strategy name,
    and parameters.
    Returns a Pandas dataframe of the processed output.
    """
    if in_row:
        output = pd.DataFrame(output).T
    output["Instrument"] = instrument
    output["Strategy"] = strategy.__name__
    output.pop("_strategy")
    output["StrategyParameters"] = strategy.__dict__
    return output


def save_output(output, output_dir, instrument, start, end):
    """
    Save backtest output to file and generate chart if specified.
    """
    print(f"Saving output for {instrument}")
    fileNameOutput = f"{output_dir}/{instrument}-{start}-{end}.csv"
    output.to_csv(fileNameOutput)


def plot_results(bt, output_dir, instrument, start, end):
    print(f"Saving chart for {instrument}")
    fileNameChart = f"{output_dir}/{instrument}-{start}-{end}.html"
    bt.plot(filename=fileNameChart, open_browser=False)

def run_backtests(instruments, strategy, num_threads=4, generate_plots=False):
    """
    Run backtests for a list of instruments using a specified strategy.
    Returns a list of Pandas dataframes of the backtest results.

    Args:
        instruments (list): List of instruments to run backtests for

    Returns:
        List of Pandas dataframes of the backtest results
    """
    outputs = []
    output_dir = f"output/raw/{strategy.__name__}"
    output_dir_charts = f"output/charts/{strategy.__name__}"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if not os.path.exists(output_dir_charts):
        os.makedirs(output_dir_charts)
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        future_to_instrument = {
            executor.submit(process_instrument, instrument, strategy): instrument
            for instrument in instruments
        }
        for future in concurrent.futures.as_completed(future_to_instrument):
            instrument = future_to_instrument[future]
            output = future.result()
            if output is not None:
                outputs.append(output[0])
                save_output(output[0], output_dir, instrument, output[0]["Start"].to_string().strip().split()[1], output[0]["End"].to_string().strip().split()[1])
                if generate_plots:
                    plot_results(output[1], output_dir_charts, instrument, output[0]["Start"].to_string().strip().split()[1], output[0]["End"].to_string().strip().split()[1])
    data_frame = pd.concat(outputs)
    start = data_frame["Start"].to_string().strip().split()[1]
    end = data_frame["End"].to_string().strip().split()[1]
    fileNameOutput = f"output/{strategy.__name__}-{start}-{end}.csv"
    data_frame.to_csv(fileNameOutput)


    return data_frame


def run_backtests_optimise(instruments, strategy, num_threads=4, generate_plots=False):
    """
    Run backtests for a list of instruments using a specified strategy.
    Returns a list of Pandas dataframes of the backtest results.
    """
    outputs = []
    output_dir = f"output/raw/{strategy.__name__}"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        future_to_instrument = {
            executor.submit(process_instrument_optimise, instrument, strategy): instrument
            for instrument in track(instruments)
        }
        for future in track(concurrent.futures.as_completed(future_to_instrument)):
            instrument = future_to_instrument[future]
            output = future.result()
            if output is not None:
                outputs.append(output[0])
                save_output(output[0], output_dir, instrument, output[0]["Start"].to_string().strip().split()[1], output[0]["End"].to_string().strip().split()[1])
    data_frame = pd.concat(outputs)
    start = data_frame["Start"].to_string().strip().split()[1]
    end = data_frame["End"].to_string().strip().split()[1]
    fileNameOutput = f"output/{strategy.__name__}-{start}-{end}.csv"
    if generate_plots:
        plot_results(output[1], output_dir, instrument, start, end)

    return data_frame

In [7]:
tickers = get_SP500()
run_backtests(tickers, strategy=EMA, num_threads=8, generate_plots=True)

Saving output for AOS
Saving chart for AOS
Saving output for ADM
Saving chart for ADM
Saving output for MMM
Saving chart for MMM
Saving output for ABT
Saving chart for ABT
Saving output for ATVI
Saving chart for ATVI
Saving output for ABBV
Saving chart for ABBV
Saving output for ACN
Saving chart for ACN
Saving output for ADBE
Saving chart for ADBE
Saving output for ADP
Saving chart for ADP
Saving output for AAP
Saving chart for AAP
Saving output for AES
Saving chart for AES
Saving output for APD
Saving chart for APD
Saving output for A
Saving chart for A
Saving output for ALK
Saving chart for ALK
Saving output for AFL
Saving chart for AFL
Saving output for AKAM
Saving chart for AKAM
Saving output for ALB
Saving chart for ALB
Saving output for ARE
Saving chart for ARE
Saving output for ALGN
Saving chart for ALGN
Saving output for ALLE
Saving chart for ALLE
Saving output for LNT
Saving chart for LNT
Saving output for GOOGL
Saving chart for GOOGL
Saving output for GOOG
Saving chart for GO

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],...,Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,_equity_curve,_trades,Instrument,Strategy,StrategyParameters
0,2018-04-04,2023-04-03,1825 days,53.375695,105241.02664,114257.76904,5.241027,6.94315,1.027717,22.502553,...,138 days,65 days,1.236183,1.118004,0.115825,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...,AOS,EMA,NaN
0,2018-04-04,2023-04-03,1825 days,49.245433,93213.9512,102318.76956,-6.786049,83.189066,-1.396726,21.123148,...,149 days,75 days,1.015348,0.072211,-0.206903,Equity DrawdownPct DrawdownD...,Size EntryBar ExitBar EntryPrice ExitP...,ADM,EMA,NaN
0,2018-04-04,2023-04-03,1825 days,61.795075,130487.45836,172236.403,30.487458,-51.935098,5.470769,21.288762,...,310 days,94 days,1.709613,3.147668,0.516409,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...,MMM,EMA,NaN
0,2018-04-04,2023-04-03,1825 days,51.628276,60690.80652,103254.14832,-39.309193,70.854356,-9.512179,18.515996,...,309 days,86 days,0.205395,-4.229874,-2.379591,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...,ABT,EMA,NaN
0,2018-04-04,2023-04-03,1825 days,49.80143,169810.837802,205605.303584,69.810838,28.761129,11.180753,20.506654,...,187 days,61 days,2.434808,4.411728,0.947181,Equity DrawdownPct Drawdow...,Size EntryBar ExitBar EntryPrice ExitP...,ATVI,EMA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2018-04-04,2023-04-03,1825 days,35.186656,147034.42262,160814.04262,47.034423,123.437052,8.021549,25.408134,...,119 days,39 days,1.635606,3.751289,0.695237,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...,ZBRA,EMA,NaN
0,2018-04-04,2023-04-03,1825 days,42.652899,80890.01414,118382.37616,-19.109986,36.086503,-4.156128,17.491634,...,112 days,43 days,0.89602,-0.512471,-0.544362,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...,XYL,EMA,NaN
0,2018-04-04,2023-04-03,1825 days,58.697379,205011.86756,208384.78756,105.011868,19.262295,15.453028,25.128489,...,202 days,97 days,3.432944,7.688316,1.748409,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...,ZBH,EMA,NaN
0,2018-04-04,2023-04-03,1825 days,56.076251,121205.17176,153957.09676,21.205172,-44.900662,3.924393,25.555989,...,154 days,57 days,1.436545,1.920727,0.38122,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...,ZION,EMA,NaN


In [8]:
ticker = get_all_crypto()
run_backtests(ticker, strategy=EMA, num_threads=8, generate_plots=True)

Error processing EOSETH: 'historical'
Error processing LTCBTC: 'historical'
Error processing BNTETH: 'historical'
Error processing QTUMETH: 'historical'
Error processing ETHBTC: 'historical'
Error processing SNTETH: 'historical'
Error processing BNBBTC: 'historical'
Error processing NEOBTC: 'historical'
Error processing BCCBTC: 'historical'
Error processing GASBTC: 'historical'
Error processing BNBETH: 'historical'
Error processing BTCUSDT: 'historical'
Error processing ETHUSDT: 'historical'
Error processing DNTETH: 'historical'
Error processing OAXETH: 'historical'
Error processing HSRBTC: 'historical'
Error processing ICNETH: 'historical'
Error processing MCOETH: 'historical'
Error processing MCOBTC: 'historical'
Error processing WTCETH: 'historical'
Error processing LRCBTC: 'historical'
Error processing WTCBTC: 'historical'
Error processing LRCETH: 'historical'
Error processing QTUMBTC: 'historical'
Error processing YOYOBTC: 'historical'
Error processing OMGBTC: 'historical'
Error p

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],...,Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,_equity_curve,_trades,Instrument,Strategy,StrategyParameters
0,2019-06-18,2023-03-31,1382 days,18.872017,393782.593444,393782.593444,293.782593,210.94394,43.58184,48.532922,...,61 days,23 days,6.421473,15.2527,2.629925,Equity DrawdownPct Drawdow...,Size EntryBar ExitBar EntryPrice ...,BTCBUSD,EMA,NaN
0,2019-01-31,2023-03-30,1519 days,17.171053,74.426375,205496.79663,-99.925574,30.991736,-82.266158,15.099735,...,77 days,10 days,0.712491,-3.200965,-0.885432,Equity DrawdownPct Drawdow...,Size EntryBar ExitBar EntryPrice ...,BTTBUSD,EMA,NaN


In [ ]:
# run_backtests(get_financial_statements_lists(), EMA, num_threads=4)

In [9]:
tickers = get_SP500()
run_backtests(tickers, strategy=RSI, num_threads=8, generate_plots=True)

Saving output for ADBE
Saving chart for ADBE
Saving output for MMM
Saving chart for MMM
Saving output for ACN
Saving chart for ACN
Saving output for ATVI
Saving chart for ATVI
Saving output for AOS
Saving chart for AOS
Saving output for ADM
Saving chart for ADM
Saving output for ABT
Saving chart for ABT
Saving output for ABBV
Saving chart for ABBV
Saving output for AES
Saving chart for AES
Saving output for ADP
Saving chart for ADP
Saving output for AAP
Saving chart for AAP
Saving output for A
Saving chart for A
Saving output for AFL
Saving chart for AFL
Saving output for APD
Saving chart for APD
Saving output for ALK
Saving chart for ALK
Saving output for AKAM
Saving chart for AKAM
Saving output for ALB
Saving chart for ALB
Saving output for ARE
Saving chart for ARE
Saving output for ALGN
Saving chart for ALGN
Saving output for LNT
Saving chart for LNT
Saving output for ALLE
Saving chart for ALLE
Saving output for GOOG
Saving chart for GOOG
Saving output for ALL
Saving chart for ALL
S

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],...,Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,_equity_curve,_trades,Instrument,Strategy,StrategyParameters
0,2018-04-04,2023-04-03,1825 days,84.352661,87391.645883,113174.736389,-12.608354,68.886914,-2.661492,33.626179,...,75 days,12 days,1.039963,0.078892,-0.246092,Equity DrawdownPct Drawdow...,Size EntryBar ExitBar EntryPrice Exit...,ADBE,RSI,NaN
0,2018-04-04,2023-04-03,1825 days,92.613185,54455.40676,103142.23204,-45.544593,-51.935098,-11.454546,22.385355,...,58 days,11 days,0.785922,-0.31593,-1.379502,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,MMM,RSI,NaN
0,2018-04-04,2023-04-03,1825 days,86.258936,73823.517469,117230.802925,-26.176483,90.598118,-5.893849,23.197901,...,110 days,13 days,0.929951,-0.118393,-0.538797,Equity DrawdownPct Drawdow...,Size EntryBar ExitBar EntryPrice Exi...,ACN,RSI,NaN
0,2018-04-04,2023-04-03,1825 days,84.511517,21007.962731,144245.50424,-78.992037,28.761129,-26.823919,24.37645,...,68 days,10 days,0.611436,-0.790049,-1.710413,Equity DrawdownPct Drawdow...,Size EntryBar ExitBar EntryPrice Exit...,ATVI,RSI,NaN
0,2018-04-04,2023-04-03,1825 days,93.725179,81159.9934,141900.10218,-18.840007,6.94315,-4.092185,28.79608,...,72 days,10 days,0.996558,-0.005364,-0.27452,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,AOS,RSI,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2018-04-04,2023-04-03,1825 days,87.212073,42262.01632,111042.85016,-57.737984,55.947085,-15.835175,17.455353,...,74 days,11 days,0.647239,-0.480342,-1.591211,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,YUM,RSI,NaN
0,2018-04-04,2023-04-03,1825 days,82.525814,35766.49695,171742.46582,-64.233503,123.437052,-18.600011,29.063993,...,61 days,9 days,0.810814,-0.422958,-0.800665,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,ZBRA,RSI,NaN
0,2018-04-04,2023-04-03,1825 days,88.721207,25495.55441,115643.91398,-74.504446,19.262295,-23.932594,21.661497,...,61 days,9 days,0.63925,-0.652963,-1.686471,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,ZBH,RSI,NaN
0,2018-04-04,2023-04-03,1825 days,84.590945,76057.380458,112821.74172,-23.94262,-44.900662,-5.330651,34.411859,...,51 days,9 days,0.992211,-0.013291,-0.434318,Equity DrawdownPct Drawdow...,Size EntryBar ExitBar EntryPrice Exit...,ZION,RSI,NaN


In [10]:
ticker = get_all_crypto()
run_backtests(ticker, strategy=RSI, num_threads=8, generate_plots=True)

Error processing NEOBTC: 'historical'
Error processing EOSETH: 'historical'
Error processing LTCBTC: 'historical'
Error processing ETHBTC: 'historical'
Error processing BNBBTC: 'historical'
Error processing QTUMETH: 'historical'
Error processing SNTETH: 'historical'
Error processing BNTETH: 'historical'
Error processing GASBTC: 'historical'
Error processing BCCBTC: 'historical'
Error processing BNBETH: 'historical'
Error processing ETHUSDT: 'historical'
Error processing OAXETH: 'historical'
Error processing DNTETH: 'historical'
Error processing HSRBTC: 'historical'
Error processing BTCUSDT: 'historical'
Error processing MCOETH: 'historical'
Error processing WTCBTC: 'historical'
Error processing MCOBTC: 'historical'
Error processing ICNETH: 'historical'
Error processing WTCETH: 'historical'
Error processing LRCBTC: 'historical'
Error processing LRCETH: 'historical'
Error processing QTUMBTC: 'historical'
Error processing YOYOBTC: 'historical'
Error processing OMGETH: 'historical'
Error p

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],...,Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,_equity_curve,_trades,Instrument,Strategy,StrategyParameters
0,2019-06-18,2023-03-31,1382 days,43.890094,30242.246438,170420.318604,-69.757754,210.94394,-27.066985,38.910209,...,37 days,6 days,0.716831,-1.097332,-0.956805,Equity DrawdownPct Drawdow...,Size EntryBar ExitBar EntryPrice ...,BTCBUSD,RSI,NaN
0,2019-01-31,2023-03-30,1519 days,50.394737,0.018955,114323.107409,-99.999981,30.991736,-97.569076,3.524437,...,28 days,5 days,0.70594,-1.624275,-1.677469,Equity DrawdownPct Drawdow...,Size EntryBar ExitBar EntryPrice ...,BTTBUSD,RSI,NaN


In [ ]:
# run_backtests(get_financial_statements_lists(), RSI, num_threads=4)

In [13]:
tickers = get_SP500()
run_backtests(tickers, strategy=LinearRegression, num_threads=12, generate_plots=True)

Saving output for ATVI
Saving chart for ATVI
Saving output for ABBV
Saving chart for ABBV
Saving output for AES
Saving chart for AES
Saving output for ADBE
Saving chart for ADBE
Saving output for ADP
Saving chart for ADP
Saving output for ABT
Saving chart for ABT
Saving output for AOS
Saving chart for AOS
Saving output for MMM
Saving chart for MMM
Saving output for ADM
Saving chart for ADM
Saving output for AAP
Saving chart for AAP
Saving output for ACN
Saving chart for ACN
Saving output for AFL
Saving chart for AFL
Saving output for ALK
Saving chart for ALK
Saving output for APD
Saving chart for APD
Saving output for A
Saving chart for A
Saving output for AKAM
Saving chart for AKAM
Saving output for ALL
Saving chart for ALL
Saving output for GOOG
Saving chart for GOOG
Saving output for GOOGL
Saving chart for GOOGL
Saving output for ALGN
Saving chart for ALGN
Saving output for ARE
Saving chart for ARE
Saving output for ALB
Saving chart for ALB
Saving output for LNT
Saving chart for LNT

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],...,Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,_equity_curve,_trades,Instrument,Strategy,StrategyParameters
0,2018-04-04,2023-04-03,1825 days,97.140588,9471.84209,134474.6906,-90.528158,28.761129,-37.608663,22.583871,...,39 days,9 days,0.528616,-0.976311,-2.085445,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,ATVI,LinearRegression,NaN
0,2018-04-04,2023-04-03,1825 days,96.505163,74008.07418,118907.50722,-25.991926,72.605982,-5.846807,23.767312,...,36 days,9 days,0.947944,-0.069132,-0.585017,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,ABBV,LinearRegression,NaN
0,2018-04-04,2023-04-03,1825 days,96.902303,56704.79866,135985.03948,-43.295201,109.130817,-10.734257,28.091068,...,38 days,9 days,0.906195,-0.153123,-0.757104,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,AES,LinearRegression,NaN
0,2018-04-04,2023-04-03,1825 days,97.378872,13344.375516,101454.95588,-86.655624,68.886914,-33.177811,24.730268,...,40 days,7 days,0.637218,-0.704195,-3.899743,Equity DrawdownPct Drawdow...,Size EntryBar ExitBar EntryPrice Exit...,ADBE,LinearRegression,NaN
0,2018-04-04,2023-04-03,1825 days,96.584591,74204.633776,111627.9453,-25.795366,85.752228,-5.796807,23.528987,...,43 days,9 days,0.945018,-0.076011,-0.54344,Equity DrawdownPct Drawdow...,Size EntryBar ExitBar EntryPrice Exit...,ADP,LinearRegression,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2018-04-04,2023-04-03,1825 days,96.664019,35824.279513,141930.80952,-64.17572,36.086503,-18.573706,24.118335,...,38 days,9 days,0.783682,-0.387534,-1.257351,Equity DrawdownPct Drawdow...,Size EntryBar ExitBar EntryPrice Exit...,XYL,LinearRegression,NaN
0,2018-04-04,2023-04-03,1825 days,97.617156,39609.689838,133613.2668,-60.39031,123.437052,-16.920017,32.907585,...,36 days,9 days,0.872151,-0.283953,-0.97229,Equity DrawdownPct Drawdow...,Size EntryBar ExitBar EntryPrice Exit...,ZBRA,LinearRegression,NaN
0,2018-04-04,2023-04-03,1825 days,95.869738,42372.47582,100288.2984,-57.627524,55.947085,-15.79119,17.475613,...,56 days,9 days,0.72036,-0.343684,-1.794648,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,YUM,LinearRegression,NaN
0,2018-04-04,2023-04-03,1825 days,96.108022,68632.798826,140601.597142,-31.367201,-44.900662,-7.257164,34.109812,...,35 days,9 days,0.974498,-0.045304,-0.477247,Equity DrawdownPct Drawdow...,Size EntryBar ExitBar EntryPrice Exit...,ZION,LinearRegression,NaN


In [14]:
tickers = get_all_crypto()
run_backtests(tickers, strategy=LinearRegression, num_threads=12, generate_plots=True)

Error processing LTCBTC: 'historical'
Error processing BTCUSDT: 'historical'
Error processing BNBBTC: 'historical'
Error processing ETHBTC: 'historical'
Error processing BCCBTC: 'historical'
Error processing NEOBTC: 'historical'
Error processing BNTETH: 'historical'
Error processing EOSETH: 'historical'
Error processing GASBTC: 'historical'
Error processing QTUMETH: 'historical'
Error processing SNTETH: 'historical'
Error processing BNBETH: 'historical'
Error processing ETHUSDT: 'historical'
Error processing OAXETH: 'historical'
Error processing DNTETH: 'historical'
Error processing HSRBTC: 'historical'
Error processing MCOETH: 'historical'
Error processing WTCBTC: 'historical'
Error processing WTCETH: 'historical'
Error processing LRCBTC: 'historical'
Error processing ICNETH: 'historical'
Error processing QTUMBTC: 'historical'
Error processing MCOBTC: 'historical'
Error processing LRCETH: 'historical'
Error processing YOYOBTC: 'historical'
Error processing OMGETH: 'historical'
Error p

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],...,Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,_equity_curve,_trades,Instrument,Strategy,StrategyParameters
0,2019-06-18,2023-03-31,1382 days,27.187274,15601.152355,121451.403396,-84.398848,210.94394,-38.756683,30.159888,...,18 days,5 days,0.576805,-2.224975,-1.450254,Equity DrawdownPct Drawdow...,Size EntryBar ExitBar EntryPrice ...,BTCBUSD,LinearRegression,NaN
0,2019-01-31,2023-03-30,1519 days,60.592105,0.0,423013.105069,-100.0,30.991736,0.0,124.052191,...,24 days,5 days,0.00803,-615.387965,-0.999625,Equity DrawdownPct DrawdownDura...,Size EntryBar ExitBar EntryP...,BTTBUSD,LinearRegression,NaN


In [ ]:
tickers = get_SP500()
#run_backtests(tickers, strategy=AroonOscillator, num_threads=4, generate_plots=True) # not working

In [ ]:
tickers = get_all_crypto()
#run_backtests(tickers, strategy=AroonOscillator, num_threads=4, generate_plots=True) # not working

In [15]:
tickers = get_SP500()
run_backtests(tickers, strategy=MACD, num_threads=12, generate_plots=True)

Saving output for MMM
Saving chart for MMM
Saving output for AFL
Saving chart for AFL
Saving output for ABT
Saving chart for ABT
Saving output for ABBV
Saving chart for ABBV
Saving output for AAP
Saving chart for AAP
Saving output for ADP
Saving chart for ADP
Saving output for ACN
Saving chart for ACN
Saving output for AOS
Saving chart for AOS
Saving output for ADBE
Saving chart for ADBE
Saving output for AES
Saving chart for AES
Saving output for ADM
Saving chart for ADM
Saving output for A
Saving chart for A
Saving output for ATVI
Saving chart for ATVI
Saving output for APD
Saving chart for APD
Saving output for AKAM
Saving chart for AKAM
Saving output for ALB
Saving chart for ALB
Saving output for ALK
Saving chart for ALK
Saving output for ALLE
Saving chart for ALLE
Saving output for LNT
Saving chart for LNT
Saving output for ALGN
Saving chart for ALGN
Saving output for ARE
Saving chart for ARE
Saving output for GOOGL
Saving chart for GOOGL
Saving output for ALL
Saving chart for ALL

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],...,Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,_equity_curve,_trades,Instrument,Strategy,StrategyParameters
0,2018-04-04,2023-04-03,1825 days,91.501191,45445.48206,105371.71714,-54.554518,-51.935098,-14.60278,20.912665,...,141 days,42 days,0.517261,-1.733011,-1.861268,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...,MMM,MACD,NaN
0,2018-04-04,2023-04-03,1825 days,84.193805,54052.50628,114927.3453,-45.947494,46.157324,-11.586064,19.099344,...,116 days,47 days,0.648423,-1.456478,-1.217123,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...,AFL,MACD,NaN
0,2018-04-04,2023-04-03,1825 days,93.486894,61544.21016,116322.19622,-38.45579,70.854356,-9.258918,22.877997,...,153 days,55 days,0.559914,-1.376047,-1.510829,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...,ABT,MACD,NaN
0,2018-04-04,2023-04-03,1825 days,83.637808,84576.038672,122318.716352,-15.423961,72.605982,-3.297453,23.110764,...,145 days,48 days,0.972568,-0.102195,-0.325098,Equity DrawdownPct Drawdow...,Size EntryBar ExitBar EntryPrice ExitP...,ABBV,MACD,NaN
0,2018-04-04,2023-04-03,1825 days,77.918983,41370.4149,101000.6754,-58.629585,7.171558,-16.19362,22.851679,...,122 days,43 days,0.57166,-2.181484,-1.539983,Equity DrawdownPct DrawdownD...,Size EntryBar ExitBar EntryPrice ExitP...,AAP,MACD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2018-04-04,2023-04-03,1825 days,90.627482,86963.19042,172787.11074,-13.03681,55.947085,-2.7572,19.410903,...,182 days,52 days,0.93291,-0.196946,-0.267229,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...,YUM,MACD,NaN
0,2018-04-04,2023-04-03,1825 days,59.49166,72490.9941,104900.9871,-27.509006,123.437052,-6.236326,27.493491,...,124 days,37 days,0.948685,-0.297607,-0.535084,Equity DrawdownPct DrawdownD...,Size EntryBar ExitBar EntryPrice ExitP...,ZBRA,MACD,NaN
0,2018-04-04,2023-04-03,1825 days,89.197776,104049.31788,180070.98708,4.049318,19.262295,0.797692,26.40206,...,193 days,55 days,1.165264,0.618797,0.058271,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...,ZBH,MACD,NaN
0,2018-04-04,2023-04-03,1825 days,70.532168,187506.85106,224313.4044,87.506851,102.365721,13.408811,24.912849,...,117 days,46 days,2.082926,2.87662,1.011759,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...,ZTS,MACD,NaN


In [17]:
tickers = ["BTCBUSD", "BTTBUSD"]

run_backtests(tickers, strategy=MACD, num_threads=12, generate_plots=True)

Saving output for BTTBUSD
Saving chart for BTTBUSD
Saving output for BTCBUSD
Saving chart for BTCBUSD


,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],...,Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,_equity_curve,_trades,Instrument,Strategy,StrategyParameters
0,2019-01-31,2023-03-30,1519 days,20.723684,518.718854,645605.407416,-99.481281,30.991736,-71.732785,24.567394,...,81 days,10 days,1.567495,4.491201,-0.135034,Equity DrawdownPct Drawdow...,Size EntryBar ExitBar EntryPrice ...,BTTBUSD,MACD,NaN
0,2019-06-18,2023-03-31,1382 days,37.165582,75661.146784,125652.767124,-24.338853,210.94394,-7.096461,34.426316,...,65 days,17 days,1.124676,0.766729,-0.406099,Equity DrawdownPct Drawdow...,Size EntryBar ExitBar EntryPrice ...,BTCBUSD,MACD,NaN


In [18]:
tickers = get_SP500()
run_backtests(tickers, strategy=BollingerBands, num_threads=12, generate_plots=True)

Saving output for AFL
Saving chart for AFL
Saving output for ATVI
Saving chart for ATVI
Saving output for AAP
Saving chart for AAP
Saving output for ADM
Saving chart for ADM
Saving output for AES
Saving chart for AES
Saving output for AOS
Saving chart for AOS
Saving output for ACN
Saving chart for ACN
Saving output for ADP
Saving chart for ADP
Saving output for MMM
Saving chart for MMM
Saving output for ADBE
Saving chart for ADBE
Saving output for ABBV
Saving chart for ABBV
Saving output for ABT
Saving chart for ABT
Saving output for A
Saving chart for A
Saving output for APD
Saving chart for APD
Saving output for AKAM
Saving chart for AKAM
Saving output for ALK
Saving chart for ALK
Saving output for ARE
Saving chart for ARE
Saving output for ALGN
Saving chart for ALGN
Saving output for ALL
Saving chart for ALL
Saving output for ALLE
Saving chart for ALLE
Saving output for ALB
Saving chart for ALB


In [ ]:
tickers = ["BTCBUSD", "BTTBUSD"]
run_backtests(tickers, strategy=BollingerBands, num_threads=12, generate_plots=True)

In [ ]:
tickers = get_SP500()
run_backtests(tickers, strategy=StandardDeviation, num_threads=12, generate_plots=True)

In [ ]:
tickers = ["BTCBUSD", "BTTBUSD"]
run_backtests(tickers, strategy=StandardDeviation, num_threads=12, generate_plots=True)